In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [5]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

607

In [4]:
#rest is in "starter code" for HW
foundcities = []
lat = []
temp = []
for city in cities[0:50]: # tip, only use 50 cities while you are tuning your code, then try more
    try: 
        response = requests.get(query_url + city).json()
        #print(response)
        lat.append(response['coord']['lat'])
        temp.append(response['main']['temp'])
        foundcities.append(city)
    except:
        print(f"{city} not found")
        pass
print(f"Found cities were: {foundcities}")
print(f"The latitude information received is: {lat}")
print(f"The temperature information received is: {temp}")
weather_dict = {
    "city": foundcities,
    "lat": lat,
    "temp": temp
}
weather_data = pd.DataFrame(weather_dict)

bluff not found
mahebourg not found
amderma not found
busselton not found
hermanus not found
vaini not found
atuona not found
palembang not found
xining not found
bubaque not found
carutapera not found
taolanaro not found
kandry not found
torbay not found
lagoa not found
yellowknife not found
lompoc not found
half moon bay not found
kaitangata not found
fairbanks not found
kaohsiung not found
rikitea not found
perth not found
bengkulu not found
khatanga not found
sola not found
longyearbyen not found
nagina not found
new norfolk not found
jamestown not found
basco not found
hofn not found
montreal not found
barawe not found
keti bandar not found
mataura not found
motril not found
tucupita not found
freetown not found
vangaindrano not found
ushuaia not found
rio grande not found
albany not found
castro not found
tura not found
punta arenas not found
ilulissat not found
nikolskoye not found
avarua not found
mar del plata not found
Found cities were: []
The latitude information received i